In [13]:
import pickle
import pandas as pd
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [14]:
# Load models and feature extraction
feature_bow = pickle.load(open("model/feature-bow.p", 'rb'))
model_nb = pickle.load(open('model/model-nb.p', 'rb'))
model_nn = pickle.load(open('model/model-nn.p', 'rb'))


d:\kode random\Web Bu akhsin\Streamlit\stenv\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator CountVectorizer from version 0.22.2.post1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\kode random\Web Bu akhsin\Streamlit\stenv\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MultinomialNB from version 0.22.2.post1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\kode random\Web Bu akhsin\Streamlit\stenv\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarize

In [15]:
def cleansing_with_stemming(string):
    # Convert to lowercase
    string = string.lower()
    # Remove URLs
    string = re.sub(r'http[s]?://\S+', '', string)  # Remove links
    # Remove mentions (words starting with @) and any characters that follow
    string = re.sub(r'@\w+', '', string)  # Remove mentions
    # Remove non-alphanumeric characters (except space)
    string = re.sub(r'[^a-zA-Z0-9\s]', ' ', string)
    # Remove 'rt'
    string = re.sub(r'\brt\b', '', string)  # Use word boundary \b to match 'rt' exactly
    # Remove extra spaces
    string = re.sub(r'\s+', ' ', string).strip()  # Replace multiple spaces with a single space
    
    # Remove specific suffixes: "lah", "kah", "pun"
    string = re.sub(r'\b(\w+)(lah|kah|pun)\b', r'\1', string)

    # Remove duplicate sentences
    string = ' '.join(sorted(set(string.split()), key=string.split().index))

    # Stemming with Sastrawi
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    string = stemmer.stem(string)
    
    return string


In [16]:
# Original text
original_text = '''
@FIFAWorldCup Hai FIFA ....kami Indonesia dan kami adalah tim yang paling berkembang dan energik di Asia tanpa bergantung pada wasit atau permainan kasar tidak seperti tim dari timur tengah dan anda diam melihat hal tsb tidak melakukan apapun padahal sepak bola adl. Tentang fair play,fifaworldcup hai fifa kami indonesia dan kami ada tim yang paling kembang dan energik di asia tanpa gantung pada wasit atau main kasar tidak seperti tim dari timur tengah dan anda diam lihat hal tsb tidak laku apa padahal sepak bola adl tentang fair play
'''

# Apply the cleansing and stemming
cleaned_text = cleansing_with_stemming(original_text)

# Feature extraction and predict for original text
text_feature = feature_bow.transform([cleaned_text])

print("Text: ", original_text)
print("Cleaned Text: ", cleaned_text)  # Print cleaned text for verification
print("Sentiment NB: ", model_nb.predict(text_feature)[0])
print("Sentiment NN: ", model_nn.predict(text_feature)[0])

Text:  
@FIFAWorldCup Hai FIFA ....kami Indonesia dan kami adalah tim yang paling berkembang dan energik di Asia tanpa bergantung pada wasit atau permainan kasar tidak seperti tim dari timur tengah dan anda diam melihat hal tsb tidak melakukan apapun padahal sepak bola adl. Tentang fair play,fifaworldcup hai fifa kami indonesia dan kami ada tim yang paling kembang dan energik di asia tanpa gantung pada wasit atau main kasar tidak seperti tim dari timur tengah dan anda diam lihat hal tsb tidak laku apa padahal sepak bola adl tentang fair play

Cleaned Text:  hai fifa kami indonesia dan ada tim yang paling kembang energik di asia tanpa gantung pada wasit atau main kasar tidak seperti dari timur tengah anda diam lihat hal tsb laku apa padahal sepak bola adl tentang fair play fifaworldcup kembang gantung main lihat laku
Sentiment NB:  negative
Sentiment NN:  negative


In [17]:
# Load the dataset
data = pd.read_csv('data/wasit.csv', sep=',')
print(data.head())  # Display the first few rows


   conversation_id_str                      created_at  favorite_count  \
0  1844695415763120259  Fri Oct 11 23:56:01 +0000 2024               1   
1  1844694787213148515  Fri Oct 11 23:55:47 +0000 2024               0   
2  1844531990332309949  Fri Oct 11 23:55:34 +0000 2024               0   
3  1840157006612558197  Fri Oct 11 23:55:14 +0000 2024               0   
4  1844453665631031328  Fri Oct 11 23:55:00 +0000 2024               0   

                                           full_text               id_str  \
0  @FIFAWorldCup Hai FIFA ....kami Indonesia dan ...  1844889725804609840   
1  @emhendraka Nek ra itungan ngko koyo wasit sok...  1844889666774081576   
2  @SiaranBolaLive Tak kasi sambel matanya ini wa...  1844889610125705340   
3  @fifamedia Hai FIFA ....kami Indonesia ....kam...  1844889526646472831   
4  @yellowsea_sun @idextratime Kenapa bisa lolos ...  1844889468484059456   

  image_url in_reply_to_screen_name lang              location  quote_count  \
0       NaN  

In [18]:
columns_to_drop = ['conversation_id_str', 'created_at', 'favorite_count', 'id_str', 'image_url', 
                   'in_reply_to_screen_name', 'lang', 'location', 'quote_count', 'reply_count', 
                   'retweet_count', 'tweet_url', 'user_id_str', 'username']
data = data.drop(columns=columns_to_drop, errors='ignore')

In [19]:
# Rename the column to 'Tweet' for consistency
data = data.rename(columns={'full_text': 'Tweet'})

data.head()

,Tweet
0,@FIFAWorldCup Hai FIFA ....kami Indonesia dan ...
1,@emhendraka Nek ra itungan ngko koyo wasit sok...
2,@SiaranBolaLive Tak kasi sambel matanya ini wa...
3,@fifamedia Hai FIFA ....kami Indonesia ....kam...
4,@yellowsea_sun @idextratime Kenapa bisa lolos ...


In [20]:
# Apply the cleansing and stemming to the 'Tweet' column
data['Tweet_clean'] = data['Tweet'].apply(cleansing_with_stemming)

data.head()

,Tweet,Tweet_clean
0,@FIFAWorldCup Hai FIFA ....kami Indonesia dan ...,hai fifa kami indonesia dan ada tim yang palin...
1,@emhendraka Nek ra itungan ngko koyo wasit sok...,nek ra itungan ngko koyo wasit soko om kae
2,@SiaranBolaLive Tak kasi sambel matanya ini wa...,tak kasi sambel mata ini wasit dajal biar seka...
3,@fifamedia Hai FIFA ....kami Indonesia ....kam...,hai fifa kami indonesia ada tim yang paling ke...
4,@yellowsea_sun @idextratime Kenapa bisa lolos ...,kenapa bisa lolos jadi wasit ya chicken stupid...


In [21]:
# Function to predict sentiment based on cleaned text
def predict_sentiment(sent):
    # Feature extraction
    text_feature = feature_bow.transform([sent])
    # Predict using the Naive Bayes model
    return model_nb.predict(text_feature)[0]

In [22]:
# Predict sentiment for the cleaned tweets
data['predicted_sentiment'] = data['Tweet_clean'].apply(predict_sentiment)

# Display the DataFrame with predicted sentiments
print(data[['Tweet', 'Tweet_clean', 'predicted_sentiment']].head())


                                               Tweet  \
0  @FIFAWorldCup Hai FIFA ....kami Indonesia dan ...   
1  @emhendraka Nek ra itungan ngko koyo wasit sok...   
2  @SiaranBolaLive Tak kasi sambel matanya ini wa...   
3  @fifamedia Hai FIFA ....kami Indonesia ....kam...   
4  @yellowsea_sun @idextratime Kenapa bisa lolos ...   

                                         Tweet_clean predicted_sentiment  
0  hai fifa kami indonesia dan ada tim yang palin...            negative  
1         nek ra itungan ngko koyo wasit soko om kae            negative  
2  tak kasi sambel mata ini wasit dajal biar seka...            negative  
3  hai fifa kami indonesia ada tim yang paling ke...            negative  
4  kenapa bisa lolos jadi wasit ya chicken stupid...             neutral  


In [23]:
data.head()

,Tweet,Tweet_clean,predicted_sentiment
0,@FIFAWorldCup Hai FIFA ....kami Indonesia dan ...,hai fifa kami indonesia dan ada tim yang palin...,negative
1,@emhendraka Nek ra itungan ngko koyo wasit sok...,nek ra itungan ngko koyo wasit soko om kae,negative
2,@SiaranBolaLive Tak kasi sambel matanya ini wa...,tak kasi sambel mata ini wasit dajal biar seka...,negative
3,@fifamedia Hai FIFA ....kami Indonesia ....kam...,hai fifa kami indonesia ada tim yang paling ke...,negative
4,@yellowsea_sun @idextratime Kenapa bisa lolos ...,kenapa bisa lolos jadi wasit ya chicken stupid...,neutral


In [24]:
# Save the modified DataFrame to a new CSV file
data.to_csv('data/dataset_predicted_sentiment.csv', index=False)
